In [1]:
import numpy as np
import pandas as pd
import zipfile
import io
import requests

class SSInventoryPolicy:
    """
    (s, S) inventory policy for 3-node supply chain:
    Supplier -> Warehouse -> Retailer
    """

    def __init__(
        self,
        demand_series,
        s=500,
        S=1000,
        holding_cost=1.0,
        stockout_cost=5.0,
        order_cost=50.0,
        lead_time=1,
        simulation_weeks=52,
    ):

        self.demand_series=demand_series[:simulation_weeks]
        self.s=s
        self.S=S
        self.holding_cost=holding_cost
        self.stockout_cost=stockout_cost
        self.order_cost=order_cost
        self.lead_time=lead_time
        self.simulation_weeks=simulation_weeks

        # Initialization
        self.inventory=S
        self.pipeline_orders=[]
        self.total_cost=0
        self.total_demand=0
        self.total_fulfilled=0

    def step(self,week,demand):

        # Receive pipeline orders
        if self.pipeline_orders:
            arrivals=[
                qty for (arrival_week, qty) in self.pipeline_orders
                if arrival_week==week
            ]
            for qty in arrivals:
                self.inventory+=qty

            self.pipeline_orders=[
                (arrival_week, qty)
                for (arrival_week, qty) in self.pipeline_orders
                if arrival_week>week
            ]

        # Demand occurs
        self.total_demand+=demand

        if self.inventory>=demand:
            self.inventory-=demand
            self.total_fulfilled+=demand
        else:
            self.total_fulfilled+=self.inventory
            unmet=demand-self.inventory
            self.total_cost+=unmet*self.stockout_cost
            self.inventory=0

        # Holding cost
        self.total_cost+=self.inventory*self.holding_cost

        # Reorder decision
        if self.inventory<=self.s:
            order_qty=self.S-self.inventory
            arrival_week=week+self.lead_time
            self.pipeline_orders.append((arrival_week, order_qty))
            self.total_cost+=self.order_cost

    def simulate(self):

        for week in range(self.simulation_weeks):
            demand=self.demand_series.iloc[week]
            self.step(week, demand)

        fill_rate=self.total_fulfilled/self.total_demand

        return {
            "Total Cost":round(self.total_cost, 2),
            "Fill Rate":round(fill_rate, 4),
        }


# Example usage 
if __name__ == "__main__":

    # Example: Load demand from DataCo weekly aggregated
    # Download zip from GitHub
    response=requests.get("https://raw.githubusercontent.com/SristiSarkarMCKV/Causal-RL-for-Supply-Chain-Optimization/main/data/raw/DataCoSupplyChainDataset.zip")

    # Open zip file
    zip_file=zipfile.ZipFile(io.BytesIO(response.content))

    # Load CSV inside zip
    df=pd.read_csv(zip_file.open('DataCoSupplyChainDataset.csv'), encoding='latin1')
    df["order date (DateOrders)"] = pd.to_datetime(df["order date (DateOrders)"])

    weekly_demand=(
        df.groupby(pd.Grouper(key="order date (DateOrders)", freq="W"))
        ["Order Item Quantity"]
        .sum()
        .reset_index(drop=True)
    )

    policy=SSInventoryPolicy(
        demand_series=weekly_demand,
        s=2000,
        S=5000,
        holding_cost=0.5,
        stockout_cost=10,
        order_cost=100,
        lead_time=1,
        simulation_weeks=52,
    )

    results = policy.simulate()
    print("(s, S) Policy Results\n",results)

(s, S) Policy Results
 {'Total Cost': np.float64(113259.0), 'Fill Rate': np.float64(0.9421)}
